
<a id='mccall-with-sep'></a>
<div id="qe-notebook-header" align="right" style="text-align:right;">
        <a href="https://quantecon.org/" title="quantecon.org">
                <img style="width:250px;display:inline;" width="250px" src="https://assets.quantecon.org/img/qe-menubar-logo.svg" alt="QuantEcon">
        </a>
</div>

# Job Search II: Search and Separation


<a id='index-0'></a>

## Contents

- [Job Search II: Search and Separation](#Job-Search-II:-Search-and-Separation)  
  - [Overview](#Overview)  
  - [The Model](#The-Model)  
  - [Solving the Model](#Solving-the-Model)  
  - [Implementation](#Implementation)  
  - [Impact of Parameters](#Impact-of-Parameters)  
  - [Exercises](#Exercises)  

In addition to what’s in Anaconda, this lecture will need the following libraries:

In [ ]:
!pip install quantecon

## Overview

Previously [we looked](https://python.quantecon.org/mccall_model.html) at the McCall job search model [[McCall, 1970](https://python.quantecon.org/zreferences.html#id208)] as a way of understanding unemployment and worker decisions.

One unrealistic feature of that version of the model was that every job is permanent.

In this lecture, we extend the model by introducing job separation.

Once separation enters the picture, the agent comes to view

- the loss of a job as a capital loss, and  
- a spell of unemployment as an *investment* in searching for an acceptable job  


The other minor addition is that a utility function will be included to make
worker preferences slightly more sophisticated.

We’ll need the following imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import jax
import jax.numpy as jnp
from typing import NamedTuple
from quantecon.distributions import BetaBinomial
from myst_nb import glue

## The Model

The model is similar to the [baseline McCall job search model](https://python.quantecon.org/mccall_model.html).

It concerns the life of an infinitely lived worker and

- the opportunities he or she (let’s say he to save one character) has to work at different wages  
- exogenous events that destroy his current job  
- his decision making process while unemployed  


The worker can be in one of two states: employed or unemployed.

He wants to maximize


<a id='equation-objective'></a>
$$
{\mathbb E} \sum_{t=0}^\infty \beta^t u(y_t) \tag{43.1}
$$

At this stage the only difference from the [baseline model](https://python.quantecon.org/mccall_model.html) is that we’ve added some flexibility to preferences by
introducing a utility function $ u $.

It satisfies $ u'> 0 $ and $ u'' < 0 $.

Wage offers $ \{ w_t \} $ are IID with common distribution $ q $.

The set of possible wage values is denoted by $ \mathbb W $.

### Timing and Decisions

At the start of each period, the agent can be either

- unemployed or  
- employed at some existing wage level $ w $.  


At the start of a given period, the current wage offer $ w_t $ is observed.

If currently employed, the worker

1. receives utility $ u(w) $ and  
1. is fired with some (small) probability $ \alpha $.  


If currently unemployed, the worker either accepts or rejects the current offer $ w_t $.

If he accepts, then he begins work immediately at wage $ w_t $.

If he rejects, then he receives unemployment compensation $ c $.

The process then repeats.

>**Note**
>
>We do not allow for job search while employed—this topic is taken up in a [later lecture](https://python.quantecon.org/jv.html).

## Solving the Model

We drop time subscripts in what follows and primes denote next period values.

Let

- $ v_e(w) $ be maximum lifetime value accruing to a worker who enters the current
  period *employed* with existing wage $ w $  
- $ v_u(w) $ be maximum lifetime value accruing to a worker who who enters the
  current period *unemployed* and receives wage offer $ w $.  


Here **maximum lifetime value** means the value of [(43.1)](#equation-objective) when
the worker makes optimal decisions at all future points in time.

As we now show, these obtaining these functions is key to solving the new model.

### The Bellman Equations

We recall that, in [the original job search model](https://python.quantecon.org/mccall_model.html), the
value function (the value of being unemployed with a given wage offer) satisfied
a Bellman equation.

Here this function again satisfies a Bellman equation that looks very similar.


<a id='equation-bell2-mccall'></a>
$$
v_u(w) = \max 
        \left\{ 
            v_e(w), \,  
            u(c) + \beta \sum_{w' \in \mathbb W} v_u(w') q(w') 
        \right\} \tag{43.2}
$$

The difference is that the value of accepting is $ v_e(w) $ rather than
$ w/(1-\beta) $.

We have to make this change because jobs are not permanent.

Accepting transitions the worker to employment and hence yields reward $ v_e(w) $.

Rejecting leads to unemployment compensation and unemployment tomorrow.

Equation [(43.2)](#equation-bell2-mccall) expresses the value of being unemployed with offer
$ w $ in hand as a maximum over the value of two options: accept or reject
the current offer.

The function $ v_e $ also satisfies a Bellman equation:


<a id='equation-bell1-mccall'></a>
$$
v_e(w) = u(w) + \beta
        \left[
            (1-\alpha)v_e(w) + \alpha \sum_{w' \in \mathbb W} v_u(w') q(w')
        \right] \tag{43.3}
$$

>**Note**
>
>This equation differs from a traditional Bellman equation because there is no max.

There is no max because an employed agent has no choices.

Nonetheless, in keeping with most of the literature, we also refer to it as a
Bellman equation.

Equation [(43.3)](#equation-bell1-mccall) expresses the value of being employed at wage $ w $ in terms of

- current reward $ u(w) $ plus  
- discounted expected reward tomorrow, given the $ \alpha $ probability of being fired  


As we will see, equations [(43.3)](#equation-bell1-mccall) and [(43.2)](#equation-bell2-mccall) provide
enough information to solve for both $ v_e $ and $ v_u $.

Once we have them in hand, we will be able to make optimal choices.


<a id='ast-mcm'></a>

### A Simplifying Transformation

Rather than jumping straight into solving these equations, let’s see if we can
simplify them somewhat.

(This process will be analogous to our [second pass](https://python.quantecon.org/mccall_model.html#mm-op2) at the plain vanilla
McCall model, where we reduced the Bellman equation to an equation in an unknown
scalar value, rather than an unknown vector.)

First, let


<a id='equation-defh-mm'></a>
$$
h := u(c) + \beta \sum_{w' \in \mathbb W} v_u(w') q(w') \tag{43.4}
$$

be the continuation value associated with unemployment (the value of rejecting the current offer).

We can now write [(43.2)](#equation-bell2-mccall) as

$$
v_u(w) = \max \left\{ v_e(w), \,  h \right\}
$$

or, shifting time forward one period

$$
\sum_{w' \in \mathbb W} v_u(w') q(w')
 = \sum_{w' \in \mathbb W} \max \left\{ v_e(w'), \,  h \right\} q(w')
$$

Using [(43.4)](#equation-defh-mm) again now gives


<a id='equation-bell02-mccall'></a>
$$
h = u(c) + \beta \sum_{w' \in \mathbb W} \max \left\{ v_e(w'), \,  h \right\} q(w') \tag{43.5}
$$

Finally, from [(43.4)](#equation-defh-mm) we have

$$
\sum_{w' \in \mathbb W} v_u(w') q(w') = \frac{h - u(c)}{\beta}
$$

so [(43.3)](#equation-bell1-mccall) can now be rewritten as


<a id='equation-bell01-mccall'></a>
$$
v_e(w) = u(w) + \beta
    \left[
        (1-\alpha)v_e(w) + \alpha \frac{h - u(c)}{\beta}
    \right] \tag{43.6}
$$

### Simplifying to a Single Equation

We can simplify further by solving [(43.6)](#equation-bell01-mccall) for $ v_e $ as a function of $ h $.

Rearranging [(43.6)](#equation-bell01-mccall) gives

$$
v_e(w) = u(w) + \beta(1-\alpha)v_e(w) + \alpha(h - u(c))
$$

or

$$
v_e(w) - \beta(1-\alpha)v_e(w) = u(w) + \alpha(h - u(c))
$$

Solving for $ v_e(w) $:


<a id='equation-v-e-closed'></a>
$$
v_e(w) = \frac{u(w) + \alpha(h - u(c))}{1 - \beta(1-\alpha)} \tag{43.7}
$$

Substituting this into [(43.5)](#equation-bell02-mccall) yields


<a id='equation-bell-scalar'></a>
$$
h = u(c) + \beta \sum_{w' \in \mathbb W} \max \left\{ \frac{u(w') + \alpha(h - u(c))}{1 - \beta(1-\alpha)}, \,  h \right\} q(w') \tag{43.8}
$$

This is a single scalar equation in $ h $.

### The Reservation Wage

Suppose we can use [(43.8)](#equation-bell-scalar) to solve for $ h $.

Once we have $ h $, we can obtain $ v_e $ from [(43.7)](#equation-v-e-closed).

We can then determine optimal behavior for the worker.

From [(43.2)](#equation-bell2-mccall), we see that an unemployed agent accepts current offer
$ w $ if $ v_e(w) \geq h $.

This means precisely that the value of accepting is higher than the value of rejecting.

It is clear that $ v_e $ is (at least weakly) increasing in $ w $, since the agent is never made worse off by a higher wage offer.

Hence, we can express the optimal choice as accepting wage offer $ w $ if and only if

$$
w \geq \bar w
\quad \text{where} \quad
\bar w \text{ solves } v_e(\bar w) = h
$$

### Solving the Bellman Equations

We’ll use the same iterative approach to solving the Bellman equations that we
adopted in the [first job search lecture](https://python.quantecon.org/mccall_model.html).

In this case we only need to iterate on the single scalar equation [(43.8)](#equation-bell-scalar).

The iteration rule is


<a id='equation-bell-iter'></a>
$$
h_{n+1} = u(c) + \beta \sum_{w' \in \mathbb W}
    \max \left\{ \frac{u(w') + \alpha(h_n - u(c))}{1 - \beta(1-\alpha)}, \,  h_n \right\} q(w') \tag{43.9}
$$

starting from some initial condition $ h_0 $.

Once convergence is achieved, we can compute $ v_e $ from [(43.7)](#equation-v-e-closed).

(It is possible to prove that [(43.9)](#equation-bell-iter) converges via the Banach contraction mapping theorem.)

## Implementation

Let’s implement this iterative process.

In the code, you’ll see that we use a class to store the various parameters and other
objects associated with a given model.

This helps to tidy up the code and provides an object that’s easy to pass to functions.

The default utility function is a CRRA utility function

In [ ]:
def u(c, γ):
    return (c**(1 - γ) - 1) / (1 - γ)

Also, here’s a default wage distribution, based around the BetaBinomial
distribution:

In [ ]:
n = 60                                  # n possible outcomes for w
w_default = jnp.linspace(10, 20, n)     # wages between 10 and 20
a, b = 600, 400                         # shape parameters
dist = BetaBinomial(n-1, a, b)          # distribution
q_default = jnp.array(dist.pdf())       # probabilities as a JAX array

Here’s our model class for the McCall model with separation.

In [ ]:
class Model(NamedTuple):
    α: float = 0.2              # job separation rate
    β: float = 0.98             # discount factor
    γ: float = 2.0              # utility parameter (CRRA)
    c: float = 6.0              # unemployment compensation
    w: jnp.ndarray = w_default  # wage outcome space
    q: jnp.ndarray = q_default  # probabilities over wage offers

Now we iterate until successive realizations are closer together than some small tolerance level.

We then return the current iterate as an approximate solution.

First, we define a function to compute $ v_e $ from $ h $:

In [ ]:
def compute_v_e(model, h):
    " Compute v_e from h using the closed-form expression. "
    α, β, γ, c, w, q = model
    return (u(w, γ) + α * (h - u(c, γ))) / (1 - β * (1 - α))

Now we implement the iteration on $ h $ only:

In [ ]:
def update_h(model, h):
    " One update of the scalar h. "
    α, β, γ, c, w, q = model
    v_e = compute_v_e(model, h)
    h_new = u(c, γ) + β * (jnp.maximum(v_e, h) @ q)
    return h_new

Using this iteration rule, we can write our model solver.

In [ ]:
@jax.jit
def solve_model(model, tol=1e-5, max_iter=2000):
    " Iterates to convergence on the Bellman equations. "

    def cond(loop_state):
        h, i, error = loop_state
        return jnp.logical_and(error > tol, i < max_iter)

    def update(loop_state):
        h, i, error = loop_state
        h_new = update_h(model, h)
        error_new = jnp.abs(h_new - h)
        return h_new, i + 1, error_new

    # Initialize
    h_init = u(model.c, model.γ) / (1 - model.β)
    i_init = 0
    error_init = tol + 1
    init_state = (h_init, i_init, error_init)

    final_state = jax.lax.while_loop(cond, update, init_state)
    h_final, _, _ = final_state

    # Compute v_e from the converged h
    v_e_final = compute_v_e(model, h_final)

    return v_e_final, h_final

### The Reservation Wage: First Pass

The optimal choice of the agent is summarized by the reservation wage.

As discussed above, the reservation wage is the $ \bar w $ that solves
$ v_e(\bar w) = h $ where $ h $ is the continuation value.

Let’s compare $ v_e $ and $ h $ to see what they look like.

We’ll use the default parameterizations found in the code above.

In [ ]:
model = Model()
v_e, h = solve_model(model)

fig, ax = plt.subplots()
ax.plot(model.w, v_e, 'b-', lw=2, alpha=0.7, label='$v_e$')
ax.plot(model.w, [h] * len(model.w),
        'g-', lw=2, alpha=0.7, label='$h$')
ax.set_xlim(min(model.w), max(model.w))
ax.legend()
plt.show()

The value $ v_e $ is increasing because higher $ w $ generates a higher wage flow conditional on staying employed.

The reservation wage is the $ w $ where these lines meet.

### Computing the Reservation Wage

Here’s a function `compute_reservation_wage` that takes an instance of `Model`
and returns the associated reservation wage.

In [ ]:
def compute_reservation_wage(model):
    """
    Computes the reservation wage of an instance of the McCall model
    by finding the smallest w such that v_e(w) >= h. 

    """
    # Find the first i such that v_e(w_i) >= h and return w[i]
    # If no such w exists, then w_bar is set to np.inf
    v_e, h = solve_model(model)
    accept = v_e >= h
    i = jnp.argmax(accept)   # take first accept index
    w_bar = jnp.where(jnp.any(accept), model.w[i], jnp.inf)
    return w_bar

Next we will investigate how the reservation wage varies with parameters.

## Impact of Parameters

In each instance below, we’ll show you a figure and then ask you to reproduce it in the exercises.

### The Reservation Wage and Unemployment Compensation

First, let’s look at how $ \bar w $ varies with unemployment compensation.

In the figure below, we use the default parameters in the `Model` class, apart from
c (which takes the values given on the horizontal axis)

![../_build/jupyter_execute/fb6b35bc14ca1c1aa938e901c5f3cb4db4643fda50092dc59dc3ce80bbb08947.png](../_build/jupyter_execute/fb6b35bc14ca1c1aa938e901c5f3cb4db4643fda50092dc59dc3ce80bbb08947.png)

  
As expected, higher unemployment compensation causes the worker to hold out for higher wages.

In effect, the cost of continuing job search is reduced.

### The Reservation Wage and Discounting

Next, let’s investigate how $ \bar w $ varies with the discount factor.

The next figure plots the reservation wage associated with different values of
$ \beta $

![../_build/jupyter_execute/f76dbd614c180a7ad1e080a5b4f733a708df1c6657d064c12a3459353a210d19.png](../_build/jupyter_execute/f76dbd614c180a7ad1e080a5b4f733a708df1c6657d064c12a3459353a210d19.png)

  
Again, the results are intuitive: More patient workers will hold out for higher wages.

### The Reservation Wage and Job Destruction

Finally, let’s look at how $ \bar w $ varies with the job separation rate $ \alpha $.

Higher $ \alpha $ translates to a greater chance that a worker will face termination in each period once employed.

![../_build/jupyter_execute/7b5cf35639dbbee22403aaa3d932b5d3e8faa20c9941bba4402c0b8bca110c6f.png](../_build/jupyter_execute/7b5cf35639dbbee22403aaa3d932b5d3e8faa20c9941bba4402c0b8bca110c6f.png)

  
Once more, the results are in line with our intuition.

If the separation rate is high, then the benefit of holding out for a higher wage falls.

Hence the reservation wage is lower.

## Exercises

## Exercise 43.1

Reproduce all the reservation wage figures shown above.

Regarding the values on the horizontal axis, use

In [ ]:
grid_size = 25
c_vals = jnp.linspace(2, 12, grid_size)         # unemployment compensation
β_vals = jnp.linspace(0.8, 0.99, grid_size)     # discount factors
α_vals = jnp.linspace(0.05, 0.5, grid_size)     # separation rate

## Solution

Here’s the first figure.

In [ ]:
def compute_res_wage_given_c(c):
    model = Model(c=c)
    w_bar = compute_reservation_wage(model)
    return w_bar

w_bar_vals = jax.vmap(compute_res_wage_given_c)(c_vals)

fig, ax = plt.subplots()
ax.set(xlabel='unemployment compensation', ylabel='reservation wage')
ax.plot(c_vals, w_bar_vals, label=r'$\bar w$ as a function of $c$')
ax.legend()
glue("mccall_resw_c", fig, display=False)
plt.show()

Here’s the second one.

In [ ]:
def compute_res_wage_given_beta(β):
    model = Model(β=β)
    w_bar = compute_reservation_wage(model)
    return w_bar

w_bar_vals = jax.vmap(compute_res_wage_given_beta)(β_vals)

fig, ax = plt.subplots()
ax.set(xlabel='discount factor', ylabel='reservation wage')
ax.plot(β_vals, w_bar_vals, label=r'$\bar w$ as a function of $\beta$')
ax.legend()
glue("mccall_resw_beta", fig, display=False)
plt.show()

Here’s the third.

In [ ]:
def compute_res_wage_given_alpha(α):
    model = Model(α=α)
    w_bar = compute_reservation_wage(model)
    return w_bar

w_bar_vals = jax.vmap(compute_res_wage_given_alpha)(α_vals)

fig, ax = plt.subplots()
ax.set(xlabel='separation rate', ylabel='reservation wage')
ax.plot(α_vals, w_bar_vals, label=r'$\bar w$ as a function of $\alpha$')
ax.legend()
glue("mccall_resw_alpha", fig, display=False)
plt.show()